In [1]:
# Test of the circuit from "Crossing a topological phase transition with a quantum computer" (https://arxiv.org/pdf/1910.05351.pdf)

In [2]:
from typing import Any

import sympy
import math
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import numpy as np
from cirq import GridQubit, ops
from qsgenerator.phase.analitical import construct_hamiltonian, get_theta_v, get_theta_w, get_theta_r
from qsgenerator.phase.circuits import build_ground_state_circuit, build_u1_gate, build_u_gate


In [3]:
size = 5 # size of quantum circuit excluding boundary qubits

In [4]:
g = -1 # G parameter from the paper 

In [5]:
H = construct_hamiltonian(size-2, g)
lam, V = np.linalg.eigh(H)

# ground state wavefunction
psi = V[:, 0] / np.linalg.norm(V[:, 0])

In [6]:
real, real_symbols = build_ground_state_circuit(size=size)

In [7]:
resolver = cirq.ParamResolver({'theta_v': get_theta_v(g), 'theta_w': get_theta_w(g), 'theta_r': get_theta_r(g)})

In [8]:
resolved = cirq.resolve_parameters(real, resolver)

In [9]:
final_state = cirq.final_state_vector(resolved)

In [10]:
cirq.partial_trace_of_state_vector_as_mixture(final_state, [1,2,3])

((-1.257285564548738e-08,
  array([-3.4948137e-08+0.j, -5.0000000e-01+0.j, -5.1457745e-09+0.j,
          4.9999997e-01+0.j, -3.4907121e-08+0.j, -5.0000000e-01+0.j,
          5.1047597e-09+0.j, -4.9999997e-01+0.j], dtype=complex64)),
 (0.24999959766864777,
  array([-0.4247757 +0.j,  0.2637529 +0.j,  0.42477572+0.j,  0.26375294+0.j,
          0.4247757 +0.j, -0.2637529 +0.j,  0.42477572+0.j,  0.26375294+0.j],
        dtype=complex64)),
 (0.24999961256980896,
  array([ 0.42587095+0.j, -0.26198074+0.j, -0.42587098+0.j, -0.26198074+0.j,
          0.42587098+0.j, -0.26198074+0.j,  0.42587098+0.j,  0.26198074+0.j],
        dtype=complex64)),
 (0.24999964237213135,
  array([ 0.2637529 +0.j,  0.42477566+0.j, -0.26375294+0.j,  0.42477575+0.j,
         -0.2637529 +0.j, -0.42477566+0.j, -0.26375294+0.j,  0.42477575+0.j],
        dtype=complex64)),
 (0.24999965727329254,
  array([-0.26198074+0.j, -0.42587095+0.j,  0.26198074+0.j, -0.425871  +0.j,
         -0.2619807 +0.j, -0.42587095+0.j, -0.261980

In [11]:
psi

array([ 0.35355339, -0.35355339, -0.35355339, -0.35355339, -0.35355339,
       -0.35355339, -0.35355339,  0.35355339])

In [12]:
g = -0.5
theta_w = get_theta_w(g)
theta_v = get_theta_v(g)
theta_r = get_theta_r(g)

q1, q2 = cirq.GridQubit.rect(1, 2)
cirq.unitary(build_u1_gate(q1, q2, theta_r))

array([[ 0.40824829+0.j,  0.57735027+0.j,  0.40824829+0.j,
         0.57735027+0.j],
       [ 0.57735027+0.j, -0.40824829+0.j,  0.57735027+0.j,
        -0.40824829+0.j],
       [ 0.57735027+0.j,  0.40824829+0.j, -0.57735027+0.j,
        -0.40824829+0.j],
       [-0.40824829+0.j,  0.57735027+0.j,  0.40824829+0.j,
        -0.57735027+0.j]])